In [ ]:
using LowLevelFEM
import LowLevelFEM as FEM

gmsh.initialize()

In [ ]:
gmsh.open("DINAMIC_BIG.geo")

In [ ]:
alu = FEM.material("alu", E=0.71e5, ν=0.334, ρ=2.7e-9)
vas = FEM.material("vas", E=1.96e5, ν=0.25, ρ=7.87e-9)
problem = FEM.Problem([alu, vas], type=:PlaneStress)

In [ ]:
supp = FEM.displacementConstraint("left", ux=0, uy=0)
force = FEM.load("right", fx=-1)
f = FEM.loadVector(problem, [force])

In [ ]:
K = FEM.stiffnessMatrix(problem)

In [ ]:
M = FEM.massMatrix(problem)

In [ ]:
ωmax = 2π / FEM.smallestPeriodTime(K, M)

In [ ]:
C = FEM.dampingMatrix(K, M, ωmax, β=0)

In [ ]:
u0 = FEM.initialDisplacement(problem, "alu", ux=0, uy=0)
v0 = FEM.initialVelocity(problem, "alu", vx=0, vy=0);

In [ ]:
Δt = 2 / ωmax
u1, v1, t1 = FEM.CDM(K, M, C, f, u0, v0, 5Δt, Δt)

In [ ]:
u2, v2, t2 = FEM.CDM(K, M, C, 0f, u1[:, size(u1, 2)], v1[:, size(v1, 2)], 1000Δt, Δt)
t2 .+= last(t1) + Δt

In [ ]:
S1 = FEM.solveStress(problem, u1)

In [ ]:
S2 = FEM.solveStress(problem, u2)

In [ ]:
S1 = FEM.elementsToNodes(problem, S1)

In [ ]:
S2 = FEM.elementsToNodes(problem, S2)

In [ ]:
sx1 = FEM.showDoFResults(problem, S1, :sx, t=t1)

In [ ]:
sx2 = FEM.showDoFResults(problem, S2, :sx, t=t2)

In [ ]:
nodes, steps = size(S2)
nodes ÷= 9
q = zeros(2 * nodes, steps)
for i in 1:steps
    for j in 1:nodes
        q[(j-1)*2+1, i] = S2[(j-1)*9+1, i] * v2[(j-1)*2+1, i] + S2[(j-1)*9+4, i] * v2[(j-1)*2+2, i]
        q[(j-1)*2+2, i] = S2[(j-1)*9+2, i] * v2[(j-1)*2+1, i] + S2[(j-1)*9+5, i] * v2[(j-1)*2+2, i]
    end
end
q .*= -1;

In [ ]:
q2 = FEM.showDoFResults(problem, q, :qvec, t=t2)

In [ ]:
gmsh.fltk.run()

In [ ]:
gmsh.finalize()

In [ ]:
size(S2)

In [ ]:
size(v2)